## **PORTADA**


### *Análisis de Big Data con Python*


## INTRODUCCIÓN

Este proyecto está basado en una data set donde la información de diferentes Items en Stock en el WhereHause de una empresa, cada item tiene asociado el ID de un distribuidor el cual cuenta con su propia tabla, también en el data set de los items se pueden ver el precio unitario de cada producto, los impuestos sobre dicho producto, la cantidad que hay por paquete, su color, peso unitario, tipo de producto y precio del mercado. El dataset cuenta con mas de 1800 registros y se encuentra en formato .CSV, a continuación en este proyecto se realizará una carga limpieza y preparación de los datos de dicho data set. Posteriormente utilizaremos un modelo de Machine learning para realizar predicciones sobre ciertas variables escogidas y luego ver que tan acertadas son nuestras predicciones.

### **Punto Número 2**

Cargamos todas las librerías necesarias para realizar la carga y tratamiento de los datos

In [1]:

#Punto Numero 2

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer, OneHotEncoder, MinMaxScaler
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import LinearRegression
import utilitiesP as ut

In [3]:
spark = SparkSession.builder.getOrCreate()


### **Punto Número 3 y 4**

En este paso Procedemos a Cargar los datos desde nuestra fuente, que ene este caso sería un archivo CSV, Y al mismo tiempo definimos los nombres de las Columnas y los Tipos de Datos que corresponden a cada una de ellas.

In [5]:
#Punto Numero 3 y 4


df = ut.read_data(spark,'WWStockItems.csv',
                    StructType(
                        [
                            StructField("ItemID",IntegerType(),True),
                            StructField("ItemName",StringType(),True),
                            StructField("SupplierID",IntegerType(),True),
                            StructField("ColorName",StringType(),True),
                            StructField("ItemTypeName",StringType(),True),
                            StructField("QuantitiyPerOuter",IntegerType(),True),
                            StructField("TaxRate",FloatType(),True),
                            StructField("UnitPrice",FloatType(),True),
                            StructField("RetailPrice",FloatType(),True),
                            StructField("WeightPerUnit",FloatType(),True),
                        ]
                    )
                    )

df2 = ut.read_data(spark,'WWSuppliers.csv',
                       StructType(
                            [
                            StructField("SupplierID",IntegerType(),True),
                            StructField("SupplierName",StringType(),True),
                            StructField("SupplierCategoryName",StringType(),True),
                            StructField("DeliveryMethodName",StringType(),True),
                            StructField("PaymentDays",IntegerType(),True),
                            StructField("DeliveryPostalCode",IntegerType(),True),
                            StructField("BankAccountCode",IntegerType(),True),
                        ]
                    )
                    )
df.show(700,truncate=False)
df2.show(10,truncate=False)


root
 |-- ItemID: integer (nullable = true)
 |-- ItemName: string (nullable = true)
 |-- SupplierID: integer (nullable = true)
 |-- ColorName: string (nullable = true)
 |-- ItemTypeName: string (nullable = true)
 |-- QuantitiyPerOuter: integer (nullable = true)
 |-- TaxRate: float (nullable = true)
 |-- UnitPrice: float (nullable = true)
 |-- RetailPrice: float (nullable = true)
 |-- WeightPerUnit: float (nullable = true)

root
 |-- SupplierID: integer (nullable = true)
 |-- SupplierName: string (nullable = true)
 |-- SupplierCategoryName: string (nullable = true)
 |-- DeliveryMethodName: string (nullable = true)
 |-- PaymentDays: integer (nullable = true)
 |-- DeliveryPostalCode: integer (nullable = true)
 |-- BankAccountCode: integer (nullable = true)

+------+-------------------------------------------------------------------------------------+----------+---------------+-------------------+-----------------+-------+---------+-----------+-------------+
|ItemID|ItemName               

### **Punto Número 5**

Utilizamos una de las funciones que nos brinda PySpark para obtener estadísticas básicas que nos podrían ayudar en el análisis de Datos, por ejemplo, notamos que el UnitPrice (Precio de la Unidad) tiene un promedio de 38.57 lo cual nos podría funcionar para inputar algún valor nulo.

In [6]:
#Punto Numero 5
df_description = ut.describe_dataframe(df)
df_description.show()

df_description2 = ut.describe_dataframe(df2)
df_description2.show()

+-------+-----------------+--------------------+-----------------+---------------+-------------+-----------------+------------------+------------------+------------------+------------------+
|summary|           ItemID|            ItemName|       SupplierID|      ColorName| ItemTypeName|QuantitiyPerOuter|           TaxRate|         UnitPrice|       RetailPrice|     WeightPerUnit|
+-------+-----------------+--------------------+-----------------+---------------+-------------+-----------------+------------------+------------------+------------------+------------------+
|  count|             1768|                1768|             1744|           1188|         1768|             1744|              1744|              1768|              1768|              1768|
|   mean|            884.5|                NULL|5.970183486238532|           NULL|          5.0|6.172018348623853|14.908256880733944|38.579208147471846| 57.85038478476969|  1.58348416548846|
| stddev|510.5219551269726|                NU

### **Punto Número 6**

En este caso realizamos un conteo de los valores nulos en cada columna para ver que tan completo es nuestro dataset y a su vez poder tomar decisiones  inteligentes sobre como manejar dichos valores nulos

In [7]:

#Punto 6
df_nulls_counted = ut.count_nulls(df)
df_nulls_counted.show()

+------+--------+----------+---------+------------+-----------------+-------+---------+-----------+-------------+
|ItemID|ItemName|SupplierID|ColorName|ItemTypeName|QuantitiyPerOuter|TaxRate|UnitPrice|RetailPrice|WeightPerUnit|
+------+--------+----------+---------+------------+-----------------+-------+---------+-----------+-------------+
|     0|       0|        24|      580|           0|               24|     24|        0|          0|            0|
+------+--------+----------+---------+------------+-----------------+-------+---------+-----------+-------------+



### **Punto Número 7**

Habiendo analizado los valores nulos podemos manejarlos y tomar ciertas decisiones, en este caso una de las decisiones fue eliminar los valores con un suplier ID nulo , ya que esta es nuestra llave foranea al otro data set, además 24 filas no representa una gran cantidad de datos ya que nuestro dataset es de casi 2000 parametros

In [9]:

#Punto 7
df = ut.null_handling_remove(df,["SupplierID"])
df_nulls_counted = ut.count_nulls(df)
df_nulls_counted.show()
df.show()



+------+--------+----------+---------+------------+-----------------+-------+---------+-----------+-------------+
|ItemID|ItemName|SupplierID|ColorName|ItemTypeName|QuantitiyPerOuter|TaxRate|UnitPrice|RetailPrice|WeightPerUnit|
+------+--------+----------+---------+------------+-----------------+-------+---------+-----------+-------------+
|     0|       0|         0|      580|           0|                0|      0|        0|          0|            0|
+------+--------+----------+---------+------------+-----------------+-------+---------+-----------+-------------+

+------+--------------------+----------+---------+------------+-----------------+-------+---------+-----------+-------------+
|ItemID|            ItemName|SupplierID|ColorName|ItemTypeName|QuantitiyPerOuter|TaxRate|UnitPrice|RetailPrice|WeightPerUnit|
+------+--------------------+----------+---------+------------+-----------------+-------+---------+-----------+-------------+
|     1|The Gu red shirt ...|         4|    White| 

### **Punto Número 8**

Tambien se toma la desición de rellenar los valores nulos del color con una de los colores que mas se encuentra en la columna, en este caso el Negro "Black"

In [10]:

#Punto 7
df = ut.null_handling_input(df,"ColorName","Black")
df2 = ut.null_handling_input(df2,"PaymentDays",30)
df_nulls_counted = ut.count_nulls(df)
df_nulls_counted.show()
df.show()

+------+--------+----------+---------+------------+-----------------+-------+---------+-----------+-------------+
|ItemID|ItemName|SupplierID|ColorName|ItemTypeName|QuantitiyPerOuter|TaxRate|UnitPrice|RetailPrice|WeightPerUnit|
+------+--------+----------+---------+------------+-----------------+-------+---------+-----------+-------------+
|     0|       0|         0|        0|           0|                0|      0|        0|          0|            0|
+------+--------+----------+---------+------------+-----------------+-------+---------+-----------+-------------+

+------+--------------------+----------+---------+------------+-----------------+-------+---------+-----------+-------------+
|ItemID|            ItemName|SupplierID|ColorName|ItemTypeName|QuantitiyPerOuter|TaxRate|UnitPrice|RetailPrice|WeightPerUnit|
+------+--------------------+----------+---------+------------+-----------------+-------+---------+-----------+-------------+
|     1|The Gu red shirt ...|         4|    White| 

### **Punto Número 9**

Concatenamos los 2 dataframes previamente cargados con una variable key que comparten en común y los asocia, los ponemos en una misma tabla utilizando un join.


In [11]:

join_df = ut.join_dataframe(df,df2,"SupplierID","left")
join_df.show(truncate=False)
df_nulls_counted = ut.count_nulls(join_df)
df_nulls_counted.show()


+----------+------+--------------------------------------------+---------+------------+-----------------+-------+---------+-----------+-------------+------------+--------------------+------------------+-----------+------------------+---------------+
|SupplierID|ItemID|ItemName                                    |ColorName|ItemTypeName|QuantitiyPerOuter|TaxRate|UnitPrice|RetailPrice|WeightPerUnit|SupplierName|SupplierCategoryName|DeliveryMethodName|PaymentDays|DeliveryPostalCode|BankAccountCode|
+----------+------+--------------------------------------------+---------+------------+-----------------+-------+---------+-----------+-------------+------------+--------------------+------------------+-----------+------------------+---------------+
|4         |1     |The Gu red shirt XML tag t-shirt (White) 3XS|White    |Clothing    |12               |15.0   |18.0     |26.91      |0.25         |Fabrikam    | Inc.               |Clothing Supplier |30         |30                |40351          |


### **Punto Número 10**
Para el punto número 8 se realiza una tarea un poco más compleja, preparando el dataframe par un eventual análisis de datos utilizando algoritmos de *Machine learning*,  ya que las computadores son mejores con los números convertimos cada dato que puede representar una categoría en un número único y luego en un vector que me indica de forma binaria atra vez de sus valores en cero y las posiciones del vector como diferentes datos están mas cerca o lejos de otros a nivel categórico.

In [12]:
#Punto 8
#["ItemName","ColorName","ItemTypeName","SupplierName","SupplierCategoryName","DeliveryMethodName"])
list_string = ut.get_string_cols(join_df)
enc_df = ut.encoding_categories(join_df,list_string)


enc_df.show(truncate=False)

+----------+------+--------------------------------------------+---------+------------+-----------------+-------+---------+-----------+-------------+------------+--------------------+------------------+-----------+------------------+---------------+----------------+-----------------+--------------------+--------------------+----------------------------+--------------------------+
|SupplierID|ItemID|ItemName                                    |ColorName|ItemTypeName|QuantitiyPerOuter|TaxRate|UnitPrice|RetailPrice|WeightPerUnit|SupplierName|SupplierCategoryName|DeliveryMethodName|PaymentDays|DeliveryPostalCode|BankAccountCode|ItemName_Encoded|ColorName_Encoded|ItemTypeName_Encoded|SupplierName_Encoded|SupplierCategoryName_Encoded|DeliveryMethodName_Encoded|
+----------+------+--------------------------------------------+---------+------------+-----------------+-------+---------+-----------+-------------+------------+--------------------+------------------+-----------+------------------+-

### **Punto Número 11**
Colocamos las variables independientes numericas en un vector para colocarlas en una columna con el objetivo de poder utilizarlas en un un modelo, no sin antes eliminar las variables de texto y las de ID que no utilizaremos.

In [13]:
new_df = enc_df.drop("ItemID")
new_df = new_df.drop("SupplierID")
new_df = ut.drop_string_cols(new_df,"string")

new_df = ut.vectorize_columns(new_df,"WeightPerUnit")
new_df.show(truncate=False)

Save
+-----------------+-------+---------+-----------+-------------+-----------+------------------+---------------+----------------+-----------------+--------------------+--------------------+----------------------------+--------------------------+-------------------------------------------------------------------------------------------------------------------------+
|QuantitiyPerOuter|TaxRate|UnitPrice|RetailPrice|WeightPerUnit|PaymentDays|DeliveryPostalCode|BankAccountCode|ItemName_Encoded|ColorName_Encoded|ItemTypeName_Encoded|SupplierName_Encoded|SupplierCategoryName_Encoded|DeliveryMethodName_Encoded|features_vec                                                                                                             |
+-----------------+-------+---------+-----------+-------------+-----------+------------------+---------------+----------------+-----------------+--------------------+--------------------+----------------------------+--------------------------+--------------------

### **Punto Número 12**
Por último comprimimos todos los valores numéricos y los representamos atravez de vectores, además cada valor del vector esta normalizado entre 0 y 1 esto quiere decir que para un algoritmo de *Machine Learning* le será mas fácil analizar dichos datos y como se relacionan, teniendo como máximo valor el 1 y mínimo el 0, teniendo una coorelación más directa.

In [14]:
df_norm = ut.normalize_numerical_data(new_df)


df_norm.show(truncate=False)

+-----------------+-------+---------+-----------+-------------+-----------+------------------+---------------+----------------+-----------------+--------------------+--------------------+----------------------------+--------------------------+-------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------+
|QuantitiyPerOuter|TaxRate|UnitPrice|RetailPrice|WeightPerUnit|PaymentDays|DeliveryPostalCode|BankAccountCode|ItemName_Encoded|ColorName_Encoded|ItemTypeName_Encoded|SupplierName_Encoded|SupplierCategoryName_Encoded|DeliveryMethodName_Encoded|features_vec                                                                                                             |features                                                                                                     

### **Punto Número 13**
 Dividimos el dataframe en 2 partes una para utilizarlos uno como modelo de entrenamiento y otro como modelo de prueba.



In [15]:
split_df_list = ut.split_dataframe(df_norm)
train_df = split_df_list[0]
test_df = split_df_list[1]

train_df.show()
test_df.show()

+-----------------+-------+---------+-----------+-------------+-----------+------------------+---------------+----------------+-----------------+--------------------+--------------------+----------------------------+--------------------------+--------------------+--------------------+
|QuantitiyPerOuter|TaxRate|UnitPrice|RetailPrice|WeightPerUnit|PaymentDays|DeliveryPostalCode|BankAccountCode|ItemName_Encoded|ColorName_Encoded|ItemTypeName_Encoded|SupplierName_Encoded|SupplierCategoryName_Encoded|DeliveryMethodName_Encoded|        features_vec|            features|
+-----------------+-------+---------+-----------+-------------+-----------+------------------+---------------+----------------+-----------------+--------------------+--------------------+----------------------------+--------------------------+--------------------+--------------------+
|                1|   15.0|     13.0|      19.44|         0.15|         14|             64847|         563215| (224,[0],[1.0])|    (6,[0],[1.0

### **Punto Número 14**
 Configuramos un modelo de Machine Learning para entrenarlo con el dataframe previamente preparado y luego utilizamos dicho modelo para realizar predicciones sobre la variable dependiente que hayamos escogido, en este caso utilizamos el modelo de Regresión Lasso




In [16]:
lasso_regression = LinearRegression(featuresCol='features',labelCol='WeightPerUnit',elasticNetParam=1.0) # Parametros featuresCol y label son importantes


model = ut.generate_ml_model(train_df,test_df,lasso_regression)
model.show()

+-----------------+-------+---------+-----------+-------------+-----------+------------------+---------------+----------------+-----------------+--------------------+--------------------+----------------------------+--------------------------+--------------------+--------------------+-------------------+
|QuantitiyPerOuter|TaxRate|UnitPrice|RetailPrice|WeightPerUnit|PaymentDays|DeliveryPostalCode|BankAccountCode|ItemName_Encoded|ColorName_Encoded|ItemTypeName_Encoded|SupplierName_Encoded|SupplierCategoryName_Encoded|DeliveryMethodName_Encoded|        features_vec|            features|         prediction|
+-----------------+-------+---------+-----------+-------------+-----------+------------------+---------------+----------------+-----------------+--------------------+--------------------+----------------------------+--------------------------+--------------------+--------------------+-------------------+
|                1|   15.0|     13.0|      19.44|         0.15|         14|       

### **Punto Número 15**
 Por último Evaluamos la Presición del modelo utilizando la metrica del error cuadratico medio  

In [17]:
evaluator = RegressionEvaluator(predictionCol='prediction',labelCol='WeightPerUnit',metricName='mse')

ut.evaluate_ml_results(evaluator,model)

Error cuadrático medio es de: 2.1721424094664506e-11
